[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ranggakd/DAIly/blob/main/ideas/regression_forecasting_metrics/Metrics_Exploration.ipynb)
[![Open Medium Story](https://img.shields.io/badge/-Open_Medium_Story-black?logo=medium)](https://medium.com/@ranggakd/forecasting-metrics-im-new-i-tried-let-s-talk-f6208c55bc3b)
[![Open DEV Post](https://img.shields.io/badge/-Open_DEV_Post-black?logo=dev.to)](https://dev.to/ranggakd/so-i-explored-forecasting-metrics-now-i-want-your-two-cents-30p0)

There is another version of this Jupyter Notebook with Google Colab compatible. More explanation available on Medium or DEV.to.

In [1]:
#@markdown *Last running requirement version*

# !python -V

# uncomment line below if you are using Linux or Google Colab
# !pip freeze | grep -w 'pandas'
# !pip freeze | grep -w 'numpy'
# !pip freeze | grep -w 'matplotlib'
# !pip freeze | grep -w 'statsmodels'
# !pip freeze | grep -w 'scikit-learn'
# !pip freeze | grep -w 'plotly'
# !pip freeze | grep -w 'mercury'

# uncomment line below if you are using Windows
# !pip freeze | findstr /C:"pandas"
# !pip freeze | findstr /C:"numpy"
# !pip freeze | findstr /C:"matplotlib"
# !pip freeze | findstr /C:"statsmodels"
# !pip freeze | findstr /C:"scikit-learn"
# !pip freeze | findstr /C:"plotly"
# !pip freeze | findstr /C:"mercury"

Python 3.11.1
pandas==2.0.3
numpy==1.25.2
matplotlib==3.7.2
matplotlib-inline==0.1.6
statsmodels==0.14.0
scikit-learn==1.3.0
plotly==5.16.0
mercury==2.3.6


## 🛈 **Note on the Dataset and Model Selection**

> **Important**: The results derived from our analysis might reflect certain biases, as they are based on a synthetic dataset and an experimental model.

Remember, always exercise caution and discernment when interpreting the results, given the nature of our dataset and model.

In [2]:
import sys
import warnings
import random
from typing import Union, Callable, Any, TypeVar
from datetime import datetime

import numpy as np
import pandas as pd
import mercury as mr
import matplotlib.pyplot as plt
from statsmodels.tsa.ar_model import AutoReg
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.exceptions import UndefinedMetricWarning
from plotly import graph_objects as go

# xkcd-styled & humor sans font installed to give semi-sarcasm graph
plt.xkcd()
%matplotlib inline

In [3]:
USE_OLD_TYPING = sys.version_info < (3, 9)

if USE_OLD_TYPING:
    from typing import Tuple as tuple

In [4]:
# constants
R2 = 'r2'
MAE = 'mae'
MSE = 'mse'
RMSE = 'rmse'
MASE = 'mase'
MAPE = 'mape'
SMAPE = 'smape'
MBD = 'mbd'
HUNDRED_PERCENTAGE = 100
PERFECT_THRESHOLD = 0
ACCEPTABLE_THRESHOLD = .05
MODERATE_THRESHOLD = .1
HIGH_THRESHOLD = .2
VERY_HIGH_THRESHOLD = .3
EXCEEDINGLY_HIGH_THRESHOLD = 1
PERFECT = "Perfect"
VERY_ACCEPTABLE = "Very Acceptable"
ACCEPTABLE = "Acceptable"
MODERATE = "Moderate"
HIGH = "High"
VERY_HIGH = "Very High"
EXCEEDINGLY_HIGH = "Exceedingly High"
R2_PERFECT_THRESHOLD = 1
R2_VERY_ACCEPTABLE_THRESHOLD = .95
R2_ACCEPTABLE_THRESHOLD = .9
R2_MODERATE_THRESHOLD = .8
R2_HIGH_THRESHOLD = .7
R2_VERY_HIGH_THRESHOLD = .5
R2_EXCEEDINGLY_HIGH_THRESHOLD = 0
NOT_EXPLAIN_VARIABILITY = "Doesn't Explain Variablity"
WORSE_THAN_MEAN_MODEL = "Worse Than Simple Mean Model"
BETTER_THAN_NAIVE_LOW = 0.1
BETTER_THAN_NAIVE_MED = 0.5
BETTER_THAN_NAIVE_HIGH = 0.9
NAIVE_THRESHOLD = 1
WORSE_THAN_NAIVE_MODEL = "Worse Than Naive Forecast Model"
EQUIVALENT_TO_NAIVE_MODEL = "Equivalent to Naive Model"
OVERESTIMATION = 'Overestimation'
UNDERESTIMATION = 'Underestimation'

In [5]:
T = TypeVar('T', bound=Callable[..., Any])

def catch_warning_decorator(func: T) -> T:
     '''Decorator to catch and print `UndefinedMetricWarning` warnings.'''
     def wrapper(*args: Any, **kwargs: Any) -> Any:
          with warnings.catch_warnings(record=True) as w:
               warnings.simplefilter('always')
               result = func(*args, **kwargs)
               if w and isinstance(w[-1].message, UndefinedMetricWarning):
                    print(f"Warning: {w[-1].message}")
          return result
     return wrapper

def autoreg_predict(train: pd.Series,
                    test: pd.Series
                    ) -> pd.Series:
    '''Predict using AutoReg with lags = 1 and trend = ct'''
    # Training the AutoReg model with lags set to 1
    model = AutoReg(train, lags=1, trend='ct')
    model = model.fit()
    # Making predictions
    predictions = model.predict(start=len(train), end=len(train)+len(test)-1)
    return predictions

def offsetmodel_predict(series: pd.Series,
                        train_size: int,
                        offset: float = .01) -> pd.Series:
    '''Predict using Offset Model with default 1% distance of the range'''
    dist = offset * (series.max()-series.min())
    predictions = series[train_size:] + dist
    return predictions

def negoffsetmodel_predict(series: pd.Series,
                        train_size: int,
                        offset: float = .01) -> pd.Series:
    '''Predict using Negative Offset Model with default 1% distance of the range'''
    dist = offset * (series.max()-series.min())
    predictions = series[train_size:] - dist
    return predictions

def randomoffsetmodel_predict(series: pd.Series,
                              train_size: int,
                              offset: float = .01) -> pd.Series:
    '''Predict using Random Offset Model with default 1% distance of the range'''
    dist = offset * (series.max() - series.min())
    # Get the test series
    test_series = series[train_size:]
    # Generate a random sign (-1 or 1) for each data point in the test series
    random_signs = [random.choice([-1, 1]) for _ in range(len(test_series))]
    predictions = test_series + dist * pd.Series(random_signs, index=test_series.index)
    return predictions

def contains_zero(arr: np.ndarray) -> bool:
    '''Check whether the array contains zero'''
    return np.any(arr == 0)

def contains_non_negative(arr: np.ndarray) -> bool:
    '''Check whether the array contains non-negative values'''
    return np.all(arr >= 0)

def contains_non_positive(arr: np.ndarray) -> bool:
    '''Check whether the array contains non-positive values'''
    return np.all(arr <= 0)

def rmse(y_true: pd.Series,
         y_pred: pd.Series) -> float:
    """Compute the Root Mean Squared Error (RMSE)."""
    return mse(y_true, y_pred) ** .5

def mase(y_true: pd.Series,
         y_pred: pd.Series,
         y_train: pd.Series) -> float:
    """Compute the Mean Absolute Scaled Error (MASE)."""
    # Compute the mean absolute error of the forecast
    og_mae = mae(y_true, y_pred)
    # Compute the mean absolute error of the naive method on the training data
    nf_mae = mae(y_train[1:], y_train[:-1])
    return og_mae / nf_mae

def smape(y_true: pd.Series,
          y_pred: pd.Series) -> float:
     """Compute the Symmetric Mean Absolute Percentage Error (sMAPE)."""
     numerator = np.abs(y_pred - y_true)
     denominator = (np.abs(y_pred) + np.abs(y_true))
     return 100 * np.mean(2 * numerator / denominator)

def mbd(y_true: pd.Series,
        y_pred: pd.Series,
        epsilon: float = 1e-10) -> tuple[str, float]:
     """Compute the Mean Bias Deviation (MBD) and its bias."""
     numerator = (y_pred - y_true)
     denominator = (y_true + epsilon)
     if np.sum(numerator) > 0 and np.sum(denominator) > 0:
          bias = OVERESTIMATION
     elif np.sum(numerator) < 0 and np.sum(denominator) > 0:
          bias = UNDERESTIMATION
     elif np.sum(numerator) > 0 and np.sum(denominator) < 0:
          bias = OVERESTIMATION
     elif np.sum(numerator) < 0 and np.sum(denominator) < 0:
          bias = UNDERESTIMATION
     return bias, 100 * np.mean(numerator / denominator)

def categorize_metrics(metric: float, y_min: float, y_max: float) -> str:
     '''Categorize standard error metrics (MAE, MSE, RMSE) into 7 categories'''
     normalized_error = metric / (y_max - y_min)
     if normalized_error == PERFECT_THRESHOLD:
          return PERFECT
     elif normalized_error <= ACCEPTABLE_THRESHOLD:
          return VERY_ACCEPTABLE
     elif normalized_error <= MODERATE_THRESHOLD:
          return ACCEPTABLE
     elif normalized_error <= HIGH_THRESHOLD:
          return MODERATE
     elif normalized_error <= VERY_HIGH_THRESHOLD:
          return HIGH
     elif normalized_error <= EXCEEDINGLY_HIGH_THRESHOLD:
          return VERY_HIGH
     else:
          return EXCEEDINGLY_HIGH

def emoji_categorize_metrics(metric: float, y_min: float, y_max: float) -> str:
     '''Categorize standard error metrics (MAE, MSE, RMSE) into 7 emoji categories'''
     normalized_error = metric / (y_max - y_min)
     if normalized_error == PERFECT_THRESHOLD:
          return '💯'
     elif normalized_error <= ACCEPTABLE_THRESHOLD:
          return '👌'
     elif normalized_error <= MODERATE_THRESHOLD:
          return '✔️'
     elif normalized_error <= HIGH_THRESHOLD:
          return '❗'
     elif normalized_error <= VERY_HIGH_THRESHOLD:
          return '❌'
     elif normalized_error <= EXCEEDINGLY_HIGH_THRESHOLD:
          return '💀'
     else:
          return '☠'

def categorize_mse(mse: float, y_min: float, y_max: float) -> str:
     '''Categorize MSE error metrics into 7 categories'''
     # Apply square root to mse before categorizing
     rmse = mse ** .5
     return categorize_metrics(rmse, y_min, y_max)

def emoji_categorize_mse(mse: float, y_min: float, y_max: float) -> str:
     '''Categorize MSE error metrics into 7 emoji categories'''
     # Apply square root to mse before categorizing
     rmse = mse ** .5
     return emoji_categorize_metrics(rmse, y_min, y_max)

def categorize_pe(pe: float) -> str:
     '''Categorize PEs error metrics into categories based on magnitude.'''
     if pe == PERFECT_THRESHOLD:
          return PERFECT
     elif pe <= ACCEPTABLE_THRESHOLD*HUNDRED_PERCENTAGE:
          return VERY_ACCEPTABLE
     elif pe <= MODERATE_THRESHOLD*HUNDRED_PERCENTAGE:
          return ACCEPTABLE
     elif pe <= HIGH_THRESHOLD*HUNDRED_PERCENTAGE:
          return MODERATE
     elif pe <= VERY_HIGH_THRESHOLD*HUNDRED_PERCENTAGE:
          return HIGH
     elif pe <= EXCEEDINGLY_HIGH_THRESHOLD*HUNDRED_PERCENTAGE:
          return VERY_HIGH
     else:
          return EXCEEDINGLY_HIGH

def emoji_categorize_pe(pe: float) -> str:
     '''Categorize PEs error metrics into emoji categories based on magnitude.'''
     if pe == PERFECT_THRESHOLD:
          return '💯'
     elif pe <= ACCEPTABLE_THRESHOLD*HUNDRED_PERCENTAGE:
          return '👌'
     elif pe <= MODERATE_THRESHOLD*HUNDRED_PERCENTAGE:
          return '✔️'
     elif pe <= HIGH_THRESHOLD*HUNDRED_PERCENTAGE:
          return '❗'
     elif pe <= VERY_HIGH_THRESHOLD*HUNDRED_PERCENTAGE:
          return '❌'
     elif pe <= EXCEEDINGLY_HIGH_THRESHOLD*HUNDRED_PERCENTAGE:
          return '💀'
     else:
          return '☠'

def categorize_r2(r2: float) -> str:
     '''Categorize R2 score into 9 categories'''
     if r2 == R2_PERFECT_THRESHOLD:
          return PERFECT
     elif r2 >= R2_VERY_ACCEPTABLE_THRESHOLD:
          return VERY_ACCEPTABLE
     elif r2 >= R2_ACCEPTABLE_THRESHOLD:
          return ACCEPTABLE
     elif r2 >= R2_MODERATE_THRESHOLD:
          return MODERATE
     elif r2 >= R2_HIGH_THRESHOLD:
          return HIGH
     elif r2 >= R2_VERY_HIGH_THRESHOLD:
          return VERY_HIGH
     elif r2 > R2_EXCEEDINGLY_HIGH_THRESHOLD:
          return EXCEEDINGLY_HIGH
     elif r2 == R2_EXCEEDINGLY_HIGH_THRESHOLD:
          return NOT_EXPLAIN_VARIABILITY
     else:
          return WORSE_THAN_MEAN_MODEL

def emoji_categorize_r2(r2: float) -> str:
     '''Categorize R2 score into 9 emoji categories'''
     if r2 == R2_PERFECT_THRESHOLD:
          return '💯'
     elif r2 >= R2_VERY_ACCEPTABLE_THRESHOLD:
          return '👌'
     elif r2 >= R2_ACCEPTABLE_THRESHOLD:
          return '✔️'
     elif r2 >= R2_MODERATE_THRESHOLD:
          return '❗'
     elif r2 >= R2_HIGH_THRESHOLD:
          return '❌'
     elif r2 >= R2_VERY_HIGH_THRESHOLD:
          return '💀'
     elif r2 > R2_EXCEEDINGLY_HIGH_THRESHOLD:
          return '☠'
     elif r2 == R2_EXCEEDINGLY_HIGH_THRESHOLD:
          return '🚫'
     else:
          return '🛑'

def categorize_mase(mase: float) -> str:
     '''Categorize MASE into 7 categories'''
     if mase == PERFECT_THRESHOLD:
          return PERFECT
     elif mase <= BETTER_THAN_NAIVE_LOW:
          return VERY_ACCEPTABLE
     elif mase <= BETTER_THAN_NAIVE_MED:
          return ACCEPTABLE
     elif mase <= BETTER_THAN_NAIVE_HIGH:
          return MODERATE
     elif mase < NAIVE_THRESHOLD:
          return HIGH
     elif mase == NAIVE_THRESHOLD:
          return EQUIVALENT_TO_NAIVE_MODEL
     else:
          return WORSE_THAN_NAIVE_MODEL

def emoji_categorize_mase(mase: float) -> str:
     '''Categorize MASE into 7 emoji categories'''
     if mase == PERFECT_THRESHOLD:
          return '💯'
     elif mase <= BETTER_THAN_NAIVE_LOW:
          return '👌'
     elif mase <= BETTER_THAN_NAIVE_MED:
          return '✔️'
     elif mase <= BETTER_THAN_NAIVE_HIGH:
          return '❗'
     elif mase < NAIVE_THRESHOLD:
          return '❌'
     elif mase == NAIVE_THRESHOLD:
          return '⚖'
     else:
          return '🤬'

@catch_warning_decorator
def display_metrics(test: pd.Series,
                    pred: pd.Series,
                    train: pd.Series,
                    y_max: Union[int, float],
                    y_min: Union[int, float]):
    '''Display all the metrics in dataframe'''
    R2_val = r2_score(test, pred)
    MAE_val = mae(test, pred)
    MSE_val = mse(test, pred)
    RMSE_val = rmse(test, pred)
    MASE_val = mase(test, pred, train)
    MAPE_val = mape(test, pred)
    sMAPE_val = smape(test, pred)
    MBD_category, MBD_val = mbd(test, pred)
    emoji_MBD_category = '📈' if MBD_category == OVERESTIMATION else '📉'
    metric_dict = {
        'Metric': [
            R2,
            MAE,
            MSE,
            RMSE,
            MASE,
            MAPE,
            SMAPE,
            MBD
        ],
        'Value': [
            R2_val,
            MAE_val,
            MSE_val,
            RMSE_val,
            MASE_val,
            MAPE_val,
            sMAPE_val,
            MBD_val
        ],
        'Category': [
            None if np.isnan(R2_val) else categorize_r2(R2_val),
            None if np.isnan(MAE_val) else categorize_metrics(MAE_val, y_min, y_max),
            None if np.isnan(MSE_val) else categorize_metrics(MSE_val, y_min, y_max),
            None if np.isnan(RMSE_val) else categorize_metrics(RMSE_val, y_min, y_max),
            None if np.isnan(MASE_val) else categorize_mase(MASE_val),
            None if np.isnan(MAPE_val) else categorize_pe(MAPE_val),
            None if np.isnan(sMAPE_val) else categorize_pe(sMAPE_val),
            None if np.isnan(MBD_val) else f'{categorize_pe(abs(MBD_val))} {MBD_category}'
        ],
        'Emoji Category': [
            '🙅‍♂️' if np.isnan(R2_val) else emoji_categorize_r2(R2_val),
            '🙅‍♂️' if np.isnan(MAE_val) else emoji_categorize_metrics(MAE_val, y_min, y_max),
            '🙅‍♂️' if np.isnan(MSE_val) else emoji_categorize_metrics(MSE_val, y_min, y_max),
            '🙅‍♂️' if np.isnan(RMSE_val) else emoji_categorize_metrics(RMSE_val, y_min, y_max),
            '🙅‍♂️' if np.isnan(MASE_val) else emoji_categorize_mase(MASE_val),
            '🙅‍♂️' if np.isnan(MAPE_val) else emoji_categorize_pe(MAPE_val),
            '🙅‍♂️' if np.isnan(sMAPE_val) else emoji_categorize_pe(sMAPE_val),
            '🙅‍♂️' if np.isnan(MBD_val) else f'{emoji_categorize_pe(abs(MBD_val))}{emoji_MBD_category}'
        ]
    }
    df = pd.DataFrame.from_dict(metric_dict)
    display(df)

def display_forecast_plotly(train: pd.Series,
                            test: pd.Series,
                            pred: pd.Series):
    '''Visualization using Plotly'''
    fig = go.Figure()
    # Plotting training data
    fig.add_trace(go.Scatter(x=train.index, y=train.values,
                        mode='lines',
                        name='Train'))
    # Plotting test data
    fig.add_trace(go.Scatter(x=test.index, y=test.values,
                        mode='markers',
                        name='Test',
                        marker=dict(color='lightgreen')))
    # Plotting prediction data
    fig.add_trace(go.Scatter(x=pred.index, y=pred.values,
                        mode='markers',
                        name='Prediction',
                        marker=dict(color='purple')))
    # Show figure
    fig.show()

def repeated_pattern(x: int) -> int:
    """
    Compute a repeating pattern for a given integer.

    The function takes an integer input, computes its modulus with 5, and then adds 1.
    The result is a number in the range [1, 5] which repeats for every consecutive block of 5 integers.

    Parameters:
    - x (int): Input integer.

    Returns:
    - int: Integer in the range [1, 5] representing the repeated pattern.
    """
    return (x % 5) + 1

<details>
    <summary>
    <strong>Metrics Exploration Graph (click to expand/collapse)</strong>
    </summary>

![metrics exploration](https://raw.githubusercontent.com/ranggakd/DAIly/main/ideas/regression_forecasting_metrics/assets/metrics_exploration.png)

</details>

---

In [6]:
app = mr.App(
    title='Regression-Forecasting Metric Dashboard',
    description='Dashboard of experimental result across different regression/forecasting metrics',
    stop_on_error=True
)

mercury.App

In [7]:
# all syntetics dataset
def numeric_widget(num: float, label: str) -> mr.Numeric:
    return mr.Numeric(value=num, min=num, max=num+1, label=label, disabled=True)

def cos_dataset(days: int, scale_factor: float = 1, const: float = 0) -> pd.Series:
    date_range = pd.date_range(datetime(1970, 1, 1), periods=days)
    x = np.linspace(0, 2 * np.pi, days)
    y = np.cos(x) * scale_factor + const
    cos_df = pd.DataFrame({'Value': y})
    cos_df.set_index(date_range, inplace=True)
    _ = numeric_widget(days, 'X')
    if scale_factor != 1:
        _ = numeric_widget(scale_factor, 'Scale Factor')
    if const != 0:
        _ = numeric_widget(const, 'Constant')
    return cos_df['Value']

def sin_dataset(days: int, 
                scale_factor: float = 1, 
                const: float = 0,
                period: float = 1,
                amplitude: float = 2) -> pd.Series:
    date_range = pd.date_range(datetime(1970, 1, 1), periods=days)
    x = np.linspace(0, amplitude * np.pi, days)
    y = np.sin(period * x) * scale_factor + const
    sin_df = pd.DataFrame({'Value': y})
    sin_df.set_index(date_range, inplace=True)
    _ = numeric_widget(days, 'X')
    if scale_factor != 1:
        _ = numeric_widget(scale_factor, 'Scale Factor')
    if const != 0:
        _ = numeric_widget(const, 'Constant')
    if period != 1:
        _ = numeric_widget(period, 'Period')
    if amplitude != 2:
        _ = numeric_widget(amplitude, 'Amplitude')
    return sin_df['Value']

def plot_linear_dataset(days: int = 100, scale_factor: float = 5, const: float = 2):
    date_range = pd.date_range(datetime(1970, 1, 1), periods=days)
    x = np.linspace(0, 2 * np.pi, days)
    y_linear_inc = scale_factor * x + const
    y_linear_dec = (-scale_factor) * x + const
    plt.plot(x, y_linear_inc, label='Increasing Linear')
    plt.plot(x, y_linear_dec, label='Decreasing Linear')
    plt.legend()
    plt.show()

def linear_dataset(days: int, scale_factor: float = 5, const: float = 2) -> pd.Series:
    date_range = pd.date_range(datetime(1970, 1, 1), periods=days)
    x = np.linspace(0, 2 * np.pi, days)
    y = scale_factor * x + const
    linear_df = pd.DataFrame({'Value': y})
    linear_df.set_index(date_range, inplace=True)
    _ = numeric_widget(days, 'X')
    _ = numeric_widget(scale_factor, 'Scale Factor')
    _ = numeric_widget(const, 'Constant')
    return linear_df['Value']

def plot_exp_dataset(days: int = 100, scale_factor: float = 2, coefficient: float = .5):
    date_range = pd.date_range(datetime(1970, 1, 1), periods=days)
    x = np.linspace(0, 2 * np.pi, days)
    y_exp_growth = scale_factor * np.exp(coefficient * x)
    y_exp_decay = scale_factor * np.exp(coefficient * (2 * np.pi - x))
    plt.plot(x, y_exp_growth, label='Exponential Growth')
    plt.plot(x, y_exp_decay, label='Exponential Decay')
    plt.legend()
    plt.show()

def exp_dataset(days: int, 
                scale_factor: float = 2, 
                coefficient: float = .5, 
                is_decreased: bool = False) -> pd.Series:
    date_range = pd.date_range(datetime(1970, 1, 1), periods=days)
    x = np.linspace(0, 2 * np.pi, days)
    if is_decreased:
        y = scale_factor * np.exp(coefficient * (2 * np.pi - x))
    else:
        y = scale_factor * np.exp(coefficient * x)
    exp_df = pd.DataFrame({'Value': y})
    exp_df.set_index(date_range, inplace=True)
    _ = numeric_widget(days, 'X')
    _ = numeric_widget(scale_factor, 'Scale Factor')
    _ = numeric_widget(coefficient, 'Coefficient')
    return exp_df['Value']
    
def plot_quad_dataset(days: int = 100, scale_factor: float = 1, power: float = 2):
    date_range = pd.date_range(datetime(1970, 1, 1), periods=days)
    x = np.linspace(0, 2 * np.pi, days)
    y_quad_inc = scale_factor * (x)**power
    y_quad_dec = -scale_factor *(x)**power
    plt.plot(x, y_quad_inc, label='Increasing Quadratic')
    plt.plot(x, y_quad_dec, label='Decreasing Quadratic')
    plt.legend()
    plt.show()    

def quad_dataset(days: int, scale_factor: float = 1, power: float = 2) -> pd.Series:
    date_range = pd.date_range(datetime(1970, 1, 1), periods=days)
    x = np.linspace(0, 2 * np.pi, days)
    y = scale_factor * (x)**power
    quad_df = pd.DataFrame({'Value': y})
    quad_df.set_index(date_range, inplace=True)
    _ = numeric_widget(days, 'X')
    _ = numeric_widget(scale_factor, 'Scale Factor')
    _ = numeric_widget(power, 'Power')
    return quad_df['Value']

def plot_log_dataset(days: int = 100, 
                     scale_factor: float = 5, 
                     const: float = 10, 
                     hshift: float = 1):
    date_range = pd.date_range(datetime(1970, 1, 1), periods=days)
    x = np.linspace(0, 2 * np.pi, days)
    y_log_inc = const + scale_factor * np.log(x + hshift)
    y_log_dec = const - scale_factor * np.log(x + hshift)
    plt.plot(x, y_log_inc, label='Increasing Logarithmic')
    plt.plot(x, y_log_dec, label='Decreasing Logarithmic')
    plt.legend()
    plt.show()

def log_dataset(days: int, 
                scale_factor: float = 5, 
                const: float = 10, 
                hshift: float = 1) -> pd.Series:
    date_range = pd.date_range(datetime(1970, 1, 1), periods=days)
    x = np.linspace(0, 2 * np.pi, days)
    y = const + scale_factor * np.log(x + hshift)
    log_df = pd.DataFrame({'Value': y})
    log_df.set_index(date_range, inplace=True)
    _ = numeric_widget(days, 'X')
    _ = numeric_widget(scale_factor, 'Scale Factor')
    _ = numeric_widget(const, 'Constant')
    _ = numeric_widget(hshift, 'Horizontal Shift')
    return log_df['Value']

def plot_sigm_dataset(days: int = 100, 
                      scale_factor: float = 1, 
                      const: float = 1,
                      upper_asymptote: float = 10,
                      power_const: float = 5):
    date_range = pd.date_range(datetime(1970, 1, 1), periods=days)
    x = np.linspace(0, 2 * np.pi, days)
    y_sigm_inc = scale_factor * upper_asymptote / (const + np.exp(-x + power_const))
    y_sigm_dec = -scale_factor * upper_asymptote / (const + np.exp(-x + power_const))
    plt.plot(x, y_sigm_inc, label='Increasing Sigmoidal')
    plt.plot(x, y_sigm_dec, label='Decreasing Sigmoidal')
    plt.legend()
    plt.show()

def sigm_dataset(days: int, 
                 scale_factor: float = 1, 
                 const: float = 1,
                 upper_asymptote: float = 10,
                 power_const: float = 5) -> pd.Series:
    date_range = pd.date_range(datetime(1970, 1, 1), periods=days)
    x = np.linspace(0, 2 * np.pi, days)
    y = scale_factor * upper_asymptote / (const + np.exp(-x + power_const))
    sigm_df = pd.DataFrame({'Value': y})
    sigm_df.set_index(date_range, inplace=True)
    _ = numeric_widget(days, 'X')
    _ = numeric_widget(scale_factor, 'Scale Factor')
    _ = numeric_widget(const, 'Constant')
    _ = numeric_widget(upper_asymptote, 'Upper Asymptote')
    _ = numeric_widget(power_const, 'Power Constant')
    return sigm_df['Value']

def outliers_dataset(days: int = 100, 
                     scale_factor: float = 1, 
                     const: float = 0,
                     period: float = 1,
                     amplitude: float = 4) -> pd.Series:
    date_range = pd.date_range(datetime(1970, 1, 1), periods=days)
    x = np.linspace(0, amplitude * np.pi, days)
    y = np.sin(period * x) * scale_factor + const
    y_extreme = np.where(np.abs(y) > .99, 1000 * y, y)
    outliers_df = pd.DataFrame({'Value': y_extreme})
    outliers_df.set_index(date_range, inplace=True)
    _ = numeric_widget(days, 'X')
    if scale_factor != 1:
        _ = numeric_widget(scale_factor, 'Scale Factor')
    if const != 0:
        _ = numeric_widget(const, 'Constant')
    if period != 1:
        _ = numeric_widget(period, 'Period')
    if amplitude != 2:
        _ = numeric_widget(amplitude, 'Amplitude')
    return outliers_df['Value']

def rep_dataset(days: 100) -> pd.Series:
    date_range = pd.date_range(datetime(1970, 1, 1), periods=days)
    x = np.arange(0, days)
    y = np.array([repeated_pattern(i) for i in x])
    df = pd.DataFrame({'Value': y})
    df.set_index(date_range, inplace=True)
    return df['Value']


In [8]:
def ts1_cos(model1: str, model2: str):
    days = 100_000
    series = cos_dataset(days=days)
    # autoreg
    train_size = len(series)-1
    train, test  = series[:train_size], series[train_size:]
    predictions = autoreg_predict(train, test)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset
    predictions = offsetmodel_predict(series, train_size)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)

def ts1_sin(model1: str, model2: str):
    days = 100_000
    series = sin_dataset(days=days)
    # autoreg
    train_size = len(series)-1
    train, test  = series[:train_size], series[train_size:]
    predictions = autoreg_predict(train, test)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset
    predictions = offsetmodel_predict(series, train_size)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)

def ts2_cos(model1: str, model2: str):
    days = 100_000
    series = cos_dataset(days=days)
    # autoreg
    train_size = len(series)-2
    train, test  = series[:train_size], series[train_size:]
    predictions = autoreg_predict(train, test)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset
    predictions = offsetmodel_predict(series, train_size)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)

def ts2_sin(model1: str, model2: str):
    days = 100_000
    series = sin_dataset(days=days)
    # autoreg
    train_size = len(series)-2
    train, test  = series[:train_size], series[train_size:]
    predictions = autoreg_predict(train, test)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset
    predictions = offsetmodel_predict(series, train_size)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)

def tm_cos(model1: str, model2: str):
    days = 100_000
    series = cos_dataset(days=days)
    # autoreg
    train_size = len(series)-50
    train, test  = series[:train_size], series[train_size:]
    predictions = autoreg_predict(train, test)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset
    predictions = offsetmodel_predict(series, train_size)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)

def tm_sin(model1: str, model2: str):
    days = 100_000
    series = sin_dataset(days=days)
    # autoreg
    train_size = len(series)-50
    train, test  = series[:train_size], series[train_size:]
    predictions = autoreg_predict(train, test)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset
    predictions = offsetmodel_predict(series, train_size)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)

def tl_cos(model1: str, model2: str):
    days = 100_000
    series = cos_dataset(days=days)
    # autoreg
    train_size = len(series)-10_000
    train, test  = series[:train_size], series[train_size:]
    predictions = autoreg_predict(train, test)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset
    predictions = offsetmodel_predict(series, train_size)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)

def tl_sin(model1: str, model2: str):
    days = 100_000
    series = sin_dataset(days=days)
    # autoreg
    train_size = len(series)-10_000
    train, test  = series[:train_size], series[train_size:]
    predictions = autoreg_predict(train, test)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset
    predictions = offsetmodel_predict(series, train_size)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)


In [9]:
def nnz_cos(model1: str, model2: str):
    days = 100
    scale_factor = 10
    const = 1
    series = cos_dataset(days=days, scale_factor=scale_factor, const=const)
    mr.Markdown(f'is the dataset contains zero: {contains_zero(series)}')
    # autoreg
    train_size = len(series)-30
    train, test  = series[:train_size], series[train_size:]
    predictions = autoreg_predict(train, test)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset
    predictions = offsetmodel_predict(series, train_size)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)

def nnz_sin(model1: str, model2: str):
    days = 100
    scale_factor = 10
    const = 1
    series = sin_dataset(days=days, scale_factor=scale_factor, const=const)
    mr.Markdown(f'is the dataset contains zero: {contains_zero(series)}')
    # autoreg
    train_size = len(series)-30
    train, test  = series[:train_size], series[train_size:]
    predictions = autoreg_predict(train, test)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset
    predictions = offsetmodel_predict(series, train_size)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)

def nrn_cos(model1: str, model2: str):
    days = 100
    scale_factor = 10
    series = cos_dataset(days=days, scale_factor=scale_factor).astype(int)
    mr.Markdown(f'is the dataset contains zero: {contains_zero(series)}')
    # autoreg
    train_size = len(series)-30
    train, test  = series[:train_size], series[train_size:]
    predictions = autoreg_predict(train, test)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset
    predictions = offsetmodel_predict(series, train_size)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)

def nrn_sin(model1: str, model2: str):
    days = 100
    scale_factor = 10
    series = sin_dataset(days=days, scale_factor=scale_factor).astype(int)
    mr.Markdown(f'is the dataset contains zero: {contains_zero(series)}')
    # autoreg
    train_size = len(series)-30
    train, test  = series[:train_size], series[train_size:]
    predictions = autoreg_predict(train, test)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset
    predictions = offsetmodel_predict(series, train_size)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)

def nnn_cos(model1: str, model2: str):
    days = 100
    scale_factor = 10
    const = -11
    series = cos_dataset(days=days, scale_factor=scale_factor, const=const)
    mr.Markdown(f'is the dataset contains non-negative: {contains_non_negative(series)}')
    # autoreg
    train_size = len(series)-30
    train, test  = series[:train_size], series[train_size:]
    predictions = autoreg_predict(train, test)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset
    predictions = offsetmodel_predict(series, train_size)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)

def nnn_sin(model1: str, model2: str):
    days = 100
    scale_factor = 10
    const = -11
    series = sin_dataset(days=days, scale_factor=scale_factor, const=const)
    mr.Markdown(f'is the dataset contains non-negative: {contains_non_negative(series)}')
    # autoreg
    train_size = len(series)-30
    train, test  = series[:train_size], series[train_size:]
    predictions = autoreg_predict(train, test)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset
    predictions = offsetmodel_predict(series, train_size)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)

def npn_cos(model1: str, model2: str):
    days = 100
    scale_factor = 10
    const = 11
    series = cos_dataset(days=days, scale_factor=scale_factor, const=const)
    mr.Markdown(f'is the dataset contains non-positive: {contains_non_positive(series)}')
    # autoreg
    train_size = len(series)-30
    train, test  = series[:train_size], series[train_size:]
    predictions = autoreg_predict(train, test)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset
    predictions = offsetmodel_predict(series, train_size)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)

def npn_sin(model1: str, model2: str):
    days = 100
    scale_factor = 10
    const = 11
    series = sin_dataset(days=days, scale_factor=scale_factor, const=const)
    mr.Markdown(f'is the dataset contains non-positive: {contains_non_positive(series)}')
    # autoreg
    train_size = len(series)-30
    train, test  = series[:train_size], series[train_size:]
    predictions = autoreg_predict(train, test)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset
    predictions = offsetmodel_predict(series, train_size)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)

def nvs_cos(model1: str, model2: str):
    days = 100
    scale_factor = 1e-6
    series = cos_dataset(days=days, scale_factor=scale_factor)
    mr.Markdown(f'Min: {series.min()}')
    mr.Markdown(f'Max: {series.max()}')
    # autoreg
    train_size = len(series)-30
    train, test  = series[:train_size], series[train_size:]
    predictions = autoreg_predict(train, test)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset
    predictions = offsetmodel_predict(series, train_size)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)

def nvs_sin(model1: str, model2: str):
    days = 100
    scale_factor = 1e-6
    series = sin_dataset(days=days, scale_factor=scale_factor)
    mr.Markdown(f'Min: {series.min()}')
    mr.Markdown(f'Max: {series.max()}')
    # autoreg
    train_size = len(series)-30
    train, test  = series[:train_size], series[train_size:]
    predictions = autoreg_predict(train, test)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset
    predictions = offsetmodel_predict(series, train_size)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)

def nvl_cos(model1: str, model2: str):
    days = 100
    scale_factor = 1e11
    series = cos_dataset(days=days, scale_factor=scale_factor)
    mr.Markdown(f'Min: {series.min()}')
    mr.Markdown(f'Max: {series.max()}')
    # autoreg
    train_size = len(series)-30
    train, test  = series[:train_size], series[train_size:]
    predictions = autoreg_predict(train, test)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset
    predictions = offsetmodel_predict(series, train_size)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)

def nvl_sin(model1: str, model2: str):
    days = 100
    scale_factor = 1e11
    series = sin_dataset(days=days, scale_factor=scale_factor)
    mr.Markdown(f'Min: {series.min()}')
    mr.Markdown(f'Max: {series.max()}')
    # autoreg
    train_size = len(series)-30
    train, test  = series[:train_size], series[train_size:]
    predictions = autoreg_predict(train, test)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset
    predictions = offsetmodel_predict(series, train_size)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)


In [10]:
def ms_cos(model1: str, model2: str):
    days = 100
    scale_factor = 100
    const = 200
    series = cos_dataset(days=days, scale_factor=scale_factor, const=const)
    mr.Markdown(f'Min: {series.min()}')
    mr.Markdown(f'Max: {series.max()}')
    _ = series.plot.hist()
    plt.show()
    # offset 1%
    train_size = len(series)-30
    train, test  = series[:train_size], series[train_size:]
    predictions = offsetmodel_predict(series, train_size)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset 10%
    predictions = offsetmodel_predict(series, train_size, .1)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)

def ms_sin(model1: str, model2: str):
    days = 100
    scale_factor = 100
    const = 200
    series = sin_dataset(days=days, scale_factor=scale_factor, const=const)
    mr.Markdown(f'Min: {series.min()}')
    mr.Markdown(f'Max: {series.max()}')
    _ = series.plot.hist()
    plt.show()
    # offset 1%
    train_size = len(series)-30
    train, test  = series[:train_size], series[train_size:]
    predictions = offsetmodel_predict(series, train_size)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset 10%
    predictions = offsetmodel_predict(series, train_size, .1)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)

def md_cos(model1: str, model2: str):
    days = 100
    scale_factor = 10
    const = 21
    series = cos_dataset(days=days, scale_factor=scale_factor, const=const)
    mr.Markdown(f'Min: {series.min()}')
    mr.Markdown(f'Max: {series.max()}')
    _ = series.plot.hist()
    plt.show()
    # offset 500%
    train_size = len(series)-30
    train, test  = series[:train_size], series[train_size:]
    predictions = offsetmodel_predict(series, train_size, 5)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset 5000%
    predictions = offsetmodel_predict(series, train_size, 50)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)

def md_sin(model1: str, model2: str):
    days = 100
    scale_factor = 10
    const = 21
    series = sin_dataset(days=days, scale_factor=scale_factor, const=const)
    mr.Markdown(f'Min: {series.min()}')
    mr.Markdown(f'Max: {series.max()}')
    _ = series.plot.hist()
    plt.show()
    # offset 500%
    train_size = len(series)-30
    train, test  = series[:train_size], series[train_size:]
    predictions = offsetmodel_predict(series, train_size, 5)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset 5000%
    predictions = offsetmodel_predict(series, train_size, 50)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)


In [11]:
def tl_inc(model1: str, model2: str):
    days = 100
    series = linear_dataset(days=days)
    plot_linear_dataset()
    # autoreg
    train_size = len(series)-30
    train, test  = series[:train_size], series[train_size:]
    predictions = autoreg_predict(train, test)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset
    predictions = offsetmodel_predict(series, train_size)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)

def tl_dec(model1: str, model2: str):
    days = 100
    scale_factor = -5
    series = linear_dataset(days=days, scale_factor=scale_factor)
    plot_linear_dataset()
    # autoreg
    train_size = len(series)-30
    train, test  = series[:train_size], series[train_size:]
    predictions = autoreg_predict(train, test)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset
    predictions = offsetmodel_predict(series, train_size)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)

def texp_inc(model1: str, model2: str):
    days = 100
    series = exp_dataset(days=days)
    plot_exp_dataset()
    # autoreg
    train_size = len(series)-30
    train, test  = series[:train_size], series[train_size:]
    predictions = autoreg_predict(train, test)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset
    predictions = offsetmodel_predict(series, train_size)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)

def texp_dec(model1: str, model2: str):
    days = 100
    series = exp_dataset(days=days, is_decreased=True)
    plot_exp_dataset()
    # autoreg
    train_size = len(series)-30
    train, test  = series[:train_size], series[train_size:]
    predictions = autoreg_predict(train, test)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset
    predictions = offsetmodel_predict(series, train_size)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)

def tq_inc(model1: str, model2: str):
    days = 100
    series = quad_dataset(days=days)
    plot_quad_dataset()
    # autoreg
    train_size = len(series)-30
    train, test  = series[:train_size], series[train_size:]
    predictions = autoreg_predict(train, test)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset
    predictions = offsetmodel_predict(series, train_size)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)

def tq_dec(model1: str, model2: str):
    days = 100
    scale_factor = -1
    series = quad_dataset(days=days, scale_factor=scale_factor)
    plot_quad_dataset()
    # autoreg
    train_size = len(series)-30
    train, test  = series[:train_size], series[train_size:]
    predictions = autoreg_predict(train, test)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset
    predictions = offsetmodel_predict(series, train_size)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)

def tlog_inc(model1: str, model2: str):
    days = 100
    series = log_dataset(days=days)
    plot_log_dataset()
    # autoreg
    train_size = len(series)-30
    train, test  = series[:train_size], series[train_size:]
    predictions = autoreg_predict(train, test)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset
    predictions = offsetmodel_predict(series, train_size)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)

def tlog_dec(model1: str, model2: str):
    days = 100
    scale_factor = -5
    series = log_dataset(days=days, scale_factor=scale_factor)
    plot_log_dataset()
    # autoreg
    train_size = len(series)-30
    train, test  = series[:train_size], series[train_size:]
    predictions = autoreg_predict(train, test)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset
    predictions = offsetmodel_predict(series, train_size)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)

def tsigm_inc(model1: str, model2: str):
    days = 100
    series = sigm_dataset(days=days)
    plot_sigm_dataset()
    # autoreg
    train_size = len(series)-30
    train, test  = series[:train_size], series[train_size:]
    predictions = autoreg_predict(train, test)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset
    predictions = offsetmodel_predict(series, train_size)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)

def tsigm_dec(model1: str, model2: str):
    days = 100
    scale_factor = -1
    series = sigm_dataset(days=days, scale_factor=scale_factor)
    plot_sigm_dataset()
    # autoreg
    train_size = len(series)-30
    train, test  = series[:train_size], series[train_size:]
    predictions = autoreg_predict(train, test)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset
    predictions = offsetmodel_predict(series, train_size)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)

def tseason(model1: str, model2: str):
    days = 365
    scale_factor = 20
    period = 4
    series = sin_dataset(days=days, scale_factor=scale_factor, period=period)
    date_range = pd.date_range(datetime(1970, 1, 1), periods=days)
    plt.plot(date_range, series)
    plt.scatter(date_range, series, color='green')
    plt.xticks(rotation=45)
    plt.title('Yearly Temperature Variation')
    plt.show()
    # autoreg
    train_size = len(series)-30
    train, test  = series[:train_size], series[train_size:]
    predictions = autoreg_predict(train, test)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset
    predictions = offsetmodel_predict(series, train_size)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)

def toutlier(model1: str, model2: str):
    days = 100
    series = outliers_dataset(days=days)
    date_range = pd.date_range(datetime(1970, 1, 1), periods=days)
    plt.plot(date_range, series)
    plt.scatter(date_range, series, color='green')
    plt.xticks(rotation=45)
    plt.title('Piecewise function with extremes')
    plt.show()
    # autoreg
    train_size = len(series)-40
    train, test  = series[:train_size], series[train_size:]
    predictions = autoreg_predict(train, test)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset
    predictions = offsetmodel_predict(series, train_size)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)

def trep(model1: str, model2: str):
    days = 100
    series = rep_dataset(days=days)
    date_range = pd.date_range(datetime(1970, 1, 1), periods=days)
    plt.plot(date_range, series)
    plt.scatter(date_range, series, color='green')
    plt.xticks(rotation=45)
    plt.title('Repeated Pattern')
    plt.show()
    # autoreg
    train_size = len(series)-40
    train, test  = series[:train_size], series[train_size:]
    predictions = autoreg_predict(train, test)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset
    predictions = offsetmodel_predict(series, train_size)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)


In [12]:
def eover_cos(model1: str, model2: str):
    days = 100
    scale_factor = 10
    series = cos_dataset(days=days, scale_factor=scale_factor)
    # offset 1%
    train_size = len(series)-30
    train, test  = series[:train_size], series[train_size:]
    predictions = offsetmodel_predict(series, train_size)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset 10%
    predictions = offsetmodel_predict(series, train_size, .1)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)

def eunder_cos(model1: str, model2: str):
    days = 100
    scale_factor = 10
    series = cos_dataset(days=days, scale_factor=scale_factor)
    # offset 1%
    train_size = len(series)-30
    train, test  = series[:train_size], series[train_size:]
    predictions = negoffsetmodel_predict(series, train_size)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset 10%
    predictions = negoffsetmodel_predict(series, train_size, .1)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)

def erandom_cos(model1: str, model2: str):
    days = 100
    scale_factor = 10
    series = cos_dataset(days=days, scale_factor=scale_factor)
    # offset 1%
    train_size = len(series)-30
    train, test  = series[:train_size], series[train_size:]
    predictions = randomoffsetmodel_predict(series, train_size)
    y_max, y_min = train.max(), train.min()
    mr.Markdown(f'**{model1}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)
    # offset 10%
    predictions = randomoffsetmodel_predict(series, train_size, .1)
    mr.Markdown(f'**{model2}**')
    display_metrics(test, predictions, train, y_max, y_min)
    display_forecast_plotly(train, test, predictions)


In [13]:
# based on branching
mr.Note(text='---')
based_on_mapping = {
    'Test Size': 'Based on Test Size',
    'Number Nature': 'Based on Number Nature',
    'Magnitude': 'Based on Magnitude',
    'Data Distribution and Patterns': 'Based on Data Distribution and Patterns',
    'Nature of Errors': 'Based on Nature of Errors'
}
based_on_choices = list(based_on_mapping.keys())
based_on_selection = mr.Select(
    value=based_on_choices[0],
    choices=based_on_choices,
    label='Based on',
    url_key='based'
)
mr.Markdown(f'## {based_on_mapping.get(based_on_selection.value)}')

---

mercury.Select

## Based on Test Size

In [14]:
# variant branching
variant_mapping = {
    'Test Size': {
        'Small (1)': 'Few data points might cause magnified errors in some metrics. (e.g., a test size of just 1 data point)',
        'Small (>1)': 'Few data points might cause magnified errors in some metrics. (e.g., a test size of 2 data points)',
        'Mid': 'A moderate amount of data points. (e.g., test size of 50 data points)',
        'Large': 'Many data points which can sometimes mask large individual errors. (e.g., test size of 10,000 data points)'
    },
    'Number Nature': {
        'Non-zero Real Numbers': 'Standard scenario. (e.g., -5 and 7)',
        'Real Numbers with Zeros': 'Important for some metrics which cannot handle zeros. (e.g., 0 and 7)',
        'Negative Numbers Only': 'Metrics could behave differently when dealing with solely negative numbers. (e.g., -5 and -7)',
        'Positive Numbers Only': 'Standard scenario. (e.g., 5 and 10)',
        'Very Small Numbers': 'Close to zero but not zero; can test for amplification of error. (e.g., 0.000001 and 0.0001)',
        'Very Large Numbers': 'Tests for potential overflow or underflow issues in metric calculation. (e.g., 10^10 and 10^11)'
    },
    'Magnitude': {
        'Same Magnitude for $y$ and $\hat{y}$': 'Tests the performance when predictions and true values are on the same scale. (e.g., 100 and 105)',
        'Different Magnitude for $y$ and $\hat{y}$': 'Investigating how metrics react when there is a noticeable scale difference between true and predicted values. (e.g., 1 vs 1000)'
    },
    'Data Distribution and Patterns': {
        'Linear Trend': 'Examining cases where there is a consistent increase or decrease in data, indicating a linear trend',
        'Exponential Growth / Decay': 'Examining cases where there is a consistent increase or decrease in data, indicating a exponential growth / decay',
        'Quadratic Trend': 'Examining cases where there is a consistent increase or decrease in data, indicating a quadratic trend',
        'Logarithmic Trend': 'Examining cases where there is a consistent increase or decrease in data, indicating a logarithmic trend',
        'Sigmoidal/Logistic Trend': 'Examining cases where there is a consistent increase or decrease in data, indicating a sigmoidal/logistic trend',
        'Seasonality': 'Occurrences of regular fluctuations in data. (e.g., sinusoidal data patterns or sales spikes during the holidays)',
        'Outliers': 'Situations where there are extreme values that might disproportionately affect metrics. (e.g., series like 1, 2, 3, 1000, 5)',
        'Repeated Patterns': 'Situations where certain patterns in the data repeat after regular intervals. (e.g., a daily temperature dataset that consistently peaks at midday and dips at midnight like 1, 2, 3, 1, 2, 3, 1, 2, 3....)'
    },
    'Nature of Errors': {
        'Systematic Overestimation': 'When predictions are consistently higher than true values. (e.g., true: 1, 2, 3; predicted: 3, 4, 5)',
        'Systematic Underestimation': 'When predictions are consistently lower than true values. (e.g., true: 3, 4, 5; predicted: 1, 2, 3)',
        'Random Errors': 'Unpredictable error patterns where predictions sometimes overshoot and sometimes undershoot the true values and none of the predicted values exactly match the true values. (e.g., for a true sequence of 1, 2, 3, 4, 5, a prediction might be 2, 1.5, 4, 3.5, 5.5)'
    }
}
variant_choices = list(variant_mapping[based_on_selection.value].keys())
variant_selection = mr.Select(
    value=variant_choices[0],
    choices=variant_choices,
    label='Variant',
    url_key='var'
)
mr.Markdown(f'### {variant_selection.value}')
mr.Markdown(f'{variant_mapping[based_on_selection.value][variant_selection.value]}')

mercury.Select

### Small (1)

Few data points might cause magnified errors in some metrics. (e.g., a test size of just 1 data point)

In [15]:
# dataset branching
dataset_mapping = {
    'Test Size': {
        'Small (1)':{
            '$\cos(x)$':{
                ('AutoReg', 'OffsetModel 1%'): ts1_cos
            },
            '$\sin(x)$':{
                ('AutoReg', 'OffsetModel 1%'): ts1_sin
            }
        },
        'Small (>1)':{
            '$\cos(x)$':{
                ('AutoReg', 'OffsetModel 1%'): ts2_cos
            },
            '$\sin(x)$':{
                ('AutoReg', 'OffsetModel 1%'): ts2_sin
            }
        },
        'Mid':{
            '$\cos(x)$':{
                ('AutoReg', 'OffsetModel 1%'): tm_cos
            },
            '$\sin(x)$':{
                ('AutoReg', 'OffsetModel 1%'): tm_sin
            }
        },
        'Large':{
            '$\cos(x)$':{
                ('AutoReg', 'OffsetModel 1%'): tl_cos
            },
            '$\sin(x)$':{
                ('AutoReg', 'OffsetModel 1%'): tl_sin
            }
        }
    },
    'Number Nature': {
        'Non-zero Real Numbers':{
            '$10 \cdot \cos(x) + 1$':{
                ('AutoReg', 'OffsetModel 1%'): nnz_cos
            },
            '$10 \cdot \sin(x) + 1$':{
                ('AutoReg', 'OffsetModel 1%'): nnz_sin
            }
        },
        'Real Numbers with Zeros':{
            'int $(10 \cdot \cos(x))$':{
                ('AutoReg', 'OffsetModel 1%'): nrn_cos
            },
            'int $(10 \cdot \sin(x))$':{
                ('AutoReg', 'OffsetModel 1%'): nrn_sin
            }
        },
        'Negative Numbers Only':{
            '$10 \cdot \cos(x) - 11$':{
                ('AutoReg', 'OffsetModel 1%'): nnn_cos
            },
            '$10 \cdot \sin(x) - 11$':{
                ('AutoReg', 'OffsetModel 1%'): nnn_sin
            }
        },
        'Positive Numbers Only':{
            '$10 \cdot \cos(x) + 11$':{
                ('AutoReg', 'OffsetModel 1%'): npn_cos
            },
            '$10 \cdot \sin(x) + 11$':{
                ('AutoReg', 'OffsetModel 1%'): npn_sin
            }
        },
        'Very Small Numbers':{
            '$1 \cdot 10^{-6} \cdot \cos(x)$':{
                ('AutoReg', 'OffsetModel 1%'): nvs_cos
            },
            '$1 \cdot 10^{-6} \cdot \sin(x)$':{
                ('AutoReg', 'OffsetModel 1%'): nvs_sin
            }
        },
        'Very Large Numbers':{
            '$1 \cdot 10^{11} \cdot \cos(x)$':{
                ('AutoReg', 'OffsetModel 1%'): nvl_cos
            },
            '$1 \cdot 10^{11} \cdot \sin(x)$':{
                ('AutoReg', 'OffsetModel 1%'): nvl_sin
            }
        }
    },
    'Magnitude': {
        'Same Magnitude for $y$ and $\hat{y}$':{
            '$100 \cdot \cos(x) + 200$':{
                ('OffsetModel 1%', 'OffsetModel 10%'): ms_cos
            },
            '$100 \cdot \sin(x) + 200$':{
                ('OffsetModel 1%', 'OffsetModel 10%'): ms_sin
            }
        },
        'Different Magnitude for $y$ and $\hat{y}$':{
            '$10 \cdot \cos(x) + 21$':{
                ('OffsetModel 500%', 'OffsetModel 5000%'): md_cos
            },
            '$10 \cdot \sin(x) + 21$':{
                ('OffsetModel 500%', 'OffsetModel 5000%'): md_sin
            }
        }
    },
    'Data Distribution and Patterns': {
        'Linear Trend':{
            '$5x + 2$':{
                ('AutoReg', 'OffsetModel 1%'): tl_inc
            },
            '$-5x + 2$':{
                ('AutoReg', 'OffsetModel 1%'): tl_dec
            }
        },
        'Exponential Growth / Decay':{
            '$2e^{0.5x}$':{
                ('AutoReg', 'OffsetModel 1%'): texp_inc
            },
            '$2e^{0.5(2\pi - x)}$':{
                ('AutoReg', 'OffsetModel 1%'): texp_dec
            }
        },
        'Quadratic Trend':{
            '$x^2$':{
                ('AutoReg', 'OffsetModel 1%'): tq_inc
            },
            '$-x^2$':{
                ('AutoReg', 'OffsetModel 1%'): tq_dec
            }
        },
        'Logarithmic Trend':{
            '$10 + 5\ln(x+1)$':{
                ('AutoReg', 'OffsetModel 1%'): tlog_inc
            },
            '$10 - 5\ln(x+1)$':{
                ('AutoReg', 'OffsetModel 1%'): tlog_dec
            }
        },
        'Sigmoidal/Logistic Trend':{
            '${10 \over 1 + e^{-x + 5}}$':{
                ('AutoReg', 'OffsetModel 1%'): tsigm_inc
            },
            '$-{10 \over 1 + e^{-x + 5}}$':{
                ('AutoReg', 'OffsetModel 1%'): tsigm_dec
            }
        },
        'Seasonality':{
            '$20 \cdot \cos(4x) + 50$':{
                ('AutoReg', 'OffsetModel 1%'): tseason
            }
        },
        'Outliers':{
            '$\\begin{cases} \sin(x) & \\text{if } \\vert\sin(x)\\vert \leq 0.99 \\\ 1000\sin(x) & '
            '\\text{if } \\vert\sin(x)\\vert > 0.99 \\end{cases}$':{
                ('AutoReg', 'OffsetModel 1%'): toutlier
            }
        },
        'Repeated Patterns':{
            '$(x \mod 5) + 1$':{
                ('AutoReg', 'OffsetModel 1%'): trep
            }
        }
    },
    'Nature of Errors': {
        'Systematic Overestimation':{
            '$10 \cdot \cos(x)$':{
                ('OffsetModel 1%', 'OffsetModel 10%'): eover_cos
            }
        },
        'Systematic Underestimation':{
            '$10 \cdot \cos(x)$':{
                ('OffsetModel 1%', 'OffsetModel 10%'): eunder_cos
            }
        },
        'Random Errors':{
            '$10 \cdot \cos(x)$':{
                ('OffsetModel 1%', 'OffsetModel 10%'): erandom_cos
            }
        }
    }
}
dataset_choices = list(dataset_mapping[based_on_selection.value][variant_selection.value].keys())
dataset_selection = mr.Select(
    value=dataset_choices[0],
    choices=dataset_choices,
    label='Dataset',
    url_key='dataset'
)
mr.Markdown(f'#### {dataset_selection.value}')

mercury.Select

#### $\cos(x)$

In [16]:
# model branching
m = dataset_mapping[based_on_selection.value][variant_selection.value][dataset_selection.value]
for model_tuple, exp in m.items(): exp(*model_tuple)

---

<details>
    <summary>
    <strong>Summary Tables (click to expand/collapse)</strong>
    </summary>

<br>

<details>
    <summary>
    <strong>User-Defined Boundaries (click to expand/collapse)</strong>
    </summary>

> **Disclaimer:** These boundaries are user-defined. They may vary based on different context.

#### Standard Error Metrics (MAE, MSE, RMSE) Categorization

| Category           | Normalized Error Range  |
|--------------------|-------------------------|
| Perfect            | Exactly 0               |
| Very Acceptable    | $0 < x \leq 0.05$     |
| Acceptable         | $0.05 < x \leq 0.1$   |
| Moderate           | $0.1 < x \leq 0.2$    |
| High               | $0.2 < x \leq 0.3$    |
| Very High          | $0.3 < x \leq 1$    |
| Exceedingly High   | $x > 1$             |

#### Percentage Error (PE) Categorization

| Category          | Error Magnitude (%) | Direction |
|-------------------|-----------------------|-----------|
| Perfect           | Exactly 0            | -         |
| Very Acceptable   | $0 < x \leq 5$    | Over/Under|
| Acceptable        | $5 < x \leq 10$   | Over/Under|
| Moderate          | $10 < x \leq 20$  | Over/Under|
| High              | $20 < x \leq 30$  | Over/Under|
| Very High         | $30 < x \leq 100$ | Over/Under|
| Exceedingly High  | $x > 100$           | Over/Under|

#### R2 Score Categorization

| Category                           | R2 Value Range    |
|------------------------------------|-------------------|
| Perfect                            | Exactly 1         |
| Very Acceptable                    | $0.95 \leq x < 1$|
| Acceptable                         | $0.9 \leq x < 0.95$|
| Moderate                           | $0.8 \leq x < 0.9$|
| High                               | $0.7 \leq x < 0.8$|
| Very High                          | $0.5 \leq x < 0.7$|
| Exceedingly High                   | $0 < x < 0.5$   |
| Doesn't Explain Variability        | Exactly 0       |
| Worse Than Simple Mean Model       | $x < 0$         |

#### MASE Categorization

| Category           | MASE Value Range  |
|--------------------|-------------------|
| Perfect            | Exactly 0         |
| Very Acceptable    | $0 < x \leq 0.1$|
| Acceptable         | $0.1 < x \leq 0.5$|
| Moderate           | $0.5 < x \leq 0.9$|
| High               | $0.9 < x \leq 1$|
| Equivalent to Naive Model          | Exactly 1         |
| Worse Than Naive Forecast Model | $x > 1$ |

#### Severity Emojis

| Metric | Emoji |
|--------|-------|
| Perfect | 💯 |
| Very Acceptable | 👌 |
| Acceptable | ✔️ |
| Moderate | ❗ |
| High | ❌ |
| Very High | 💀 |
| Exceedingly High | ☠ |
| Doesn't Explain Variability | 🚫 |
| Worse Than Simple Mean Model | 🛑 |
| Equivalent to Naive Model | ⚖ |
| Worse Than Naive Forecast Model | 🤬 |

#### Directional Emojis

| Metric | Emoji |
|--------|-------|
| Overestimation | 📈 |
| Underestimation | 📉 |
| Nan / None | 🙅‍♂️ |

</details>

<br>

<details>
    <summary>
    <strong>Disclaimer:</strong> 
    </summary>

> 1. Metrics are calculated using `sklearn.metrics`, with the exception of `MASE`, `sMAPE`, and `MBD`. Results may vary based on different implementations.
> 2. The datasets for these metrics were synthesized using mathematical formulas like sine and cosine for controlled predictability.
> 3. The controlled models used are `statsmodels.tsa.ar_model.AutoReg` and `OffsetModel`. 
> 4. `AutoReg` was chosen for its fundamental nature in *time series forecasting*, while `OffsetModel` mimics good performance by shifting test data positively (default by 1%).
> 5. This experiment focuses on **forecasting problems**, highlighting that all forecasting problems are regressions, but not vice-versa.
> 6. When using `MBD` with negative or mixed observed values, interpret results with caution. **The metric's sign** can be influenced by both **the bias direction** and **the sign of observed values**.
> 7. This experiment is not intended to serve as a rule of thumb or a best practice. Instead, it offers a glimpse into how different metrics behave on controlled models and datasets to foster a deeper understanding.
> 8. **Use insights from this exploration at your own risk.**

</details>

<br>

| Based on | Variant | Dataset | Model | R2 | MAE | MSE | RMSE | MASE | MAPE | sMAPE | MBDev |
|--|--|--|--|--|--|--|--|--|--|--|--|
| Test Size | Small=1 | $\cos(x)$ | AutoReg |🙅‍♂️|👌|👌|👌|🤬|👌|👌|👌📈|
| | | | OffsetModel |🙅‍♂️|👌|👌|👌|🤬|👌|👌|👌📈|
| | | $\sin(x)$ | AutoReg |🙅‍♂️|👌|👌|👌|🤬|☠|☠|☠📉|
| | | | OffsetModel |🙅‍♂️|👌|👌|👌|🤬|☠|☠|☠📈|
| | Small=2 | $\cos(x)$ | AutoReg |🛑|👌|👌|👌|🤬|👌|👌|👌📈|
| | | | OffsetModel |🛑|👌|👌|👌|🤬|👌|👌|👌📈|
| | | $\sin(x)$ | AutoReg |🛑|👌|👌|👌|🤬|☠|☠|☠📉|
| | | | OffsetModel |🛑|👌|👌|👌|🤬|☠|☠|☠📈|
| | Mid | $\cos(x)$ | AutoReg |🛑|👌|👌|👌|🤬|👌|👌|👌📈|
| | | | OffsetModel |🛑|👌|👌|👌|🤬|👌|👌|👌📈|
| | | $\sin(x)$ | AutoReg |🛑|👌|👌|👌|🤬|☠|💀|☠📉|
| | | | OffsetModel |🛑|👌|👌|👌|🤬|☠|☠|☠📈|
| | Large | $\cos(x)$ | AutoReg |🛑|❗|✔️|❌|🤬|👌|❌|💀📈|
| | | | OffsetModel |❗|👌|👌|👌|🤬|👌|👌|👌📈|
| | | $\sin(x)$ | AutoReg |🛑|❌|❗|❌|🤬|☠|💀|☠📉|
| | | | OffsetModel |👌|👌|👌|👌|🤬|☠|❗|☠📈|
| Number Nature | Non-zero Real Numbers | $10 \cdot \cos(x) + 1$ | AutoReg |🛑|☠|☠|☠|🤬|👌|☠|☠📈|
| | | | OffsetModel |👌|👌|👌|👌|❗|👌|✔️|👌📈|
| | | $10 \cdot \sin(x) + 1$ | AutoReg |🛑|💀|☠|💀|🤬|❗|💀|☠📉|
| | | | OffsetModel |👌|👌|👌|👌|✔️|👌|❗|👌📈|
| | Real Numbers | $\text{int}(10 \cdot \cos(x))$ | AutoReg |🛑|💀|☠|☠|🤬|☠|☠|☠📈|
| | | | OffsetModel |👌|👌|👌|👌|❗|☠|❌|☠📈|
| | | $\text{int}(10 \cdot \sin(x))$ | AutoReg |🛑|💀|☠|💀|🤬|☠|💀|☠📉|
| | | | OffsetModel |👌|👌|👌|👌|✔️|☠|❗|☠📈|
| | Negative Numbers Only | $10 \cdot \cos(x) - 11$ | AutoReg |🛑|☠|☠|☠|🤬|❗|☠|☠📈|
| | | | OffsetModel |👌|👌|👌|👌|❗|👌|✔️|👌📈|
| | | $10 \cdot \sin(x) - 11$ | AutoReg |🛑|💀|☠|💀|🤬|👌|💀|💀📉|
| | | | OffsetModel |👌|👌|👌|👌|✔️|👌|👌|👌📈|
| | Positive Numbers Only | $10 \cdot \cos(x) + 11$ | AutoReg |🛑|☠|☠|☠|🤬|👌|💀|☠📈|
| | | | OffsetModel |👌|👌|👌|👌|❗|👌|👌|👌📈|
| | | $10 \cdot \sin(x) + 11$ | AutoReg |🛑|💀|☠|💀|🤬|👌|💀|💀📉|
| | | | OffsetModel |👌|👌|👌|👌|✔️|👌|✔️|✔️📈|
| | Very Small Numbers | $1 \times 10^{-6} \cdot \cos(x)$ | AutoReg |🛑|☠|👌|☠|🤬|👌|☠|☠📈|
| | | | OffsetModel |👌|👌|👌|👌|❗|👌|❗|👌📈|
| | | $1 \times 10^{-6} \cdot \sin(x)$ | AutoReg |🛑|💀|👌|💀|🤬|☠|💀|☠📉|
| | | | OffsetModel |👌|👌|👌|👌|✔️|☠|❗|☠📈|
| | Very Large Numbers | $1 \times 10^{11} \cdot \cos(x)$ | AutoReg |🛑|☠|☠|☠|🤬|👌|☠|☠📈|
| | | | OffsetModel |👌|👌|☠|👌|❗|👌|❗|👌📈|
| | | $1 \times 10^{11} \cdot \sin(x)$ | AutoReg |🛑|💀|☠|💀|🤬|☠|💀|☠📉|
| | | | OffsetModel |👌|👌|☠|👌|✔️|☠|❗|☠📈|
| Magnitude | Same Magnitude for $y \text{ and } \hat{y}$ | $100 \cdot \cos(x) + 200$ | OffsetModel 1% |👌|👌|👌|👌|❗|👌|👌|👌📈|
| | | | OffsetModel 10% |❌|✔️|☠|✔️|🤬|👌|✔️|✔️📈|
|  |  | $100 \cdot \sin(x) + 200$ | OffsetModel 1% |👌|👌|👌|👌|✔️|👌|👌|👌📈|
| | | | OffsetModel 10% |💀|❗|☠|❗|🤬|👌|❗|❗📈|
| | Different Magnitude for $y \text{ and } \hat{y}$ | $10 \cdot \cos(x) + 21$ | OffsetModel 500% |🛑|☠|☠|☠|🤬|👌|☠|☠📈|
| | | | OffsetModel 5000% |🛑|☠|☠|☠|🤬|💀|☠|☠📈|
| |  | $10 \cdot \sin(x) + 21$ | OffsetModel 500% |🛑|☠|☠|☠|🤬|✔️|☠|☠📈|
| | | | OffsetModel 5000% |🛑|☠|☠|☠|🤬|💀|☠|☠📈|
| Data Distribution and Patterns | Linear Trend | $5x + 2$ | AutoReg |💯|👌|👌|👌|👌|👌|👌|👌📈|
| | | | OffsetModel |👌|👌|👌|👌|❌|👌|👌|👌📈|
| |  | $-5x + 2$ | AutoReg |💯|👌|👌|👌|👌|👌|👌|👌📉|
| | | | OffsetModel |👌|👌|👌|👌|❌|👌|👌|👌📈|
| | Exponential Growth/Decay | $2e^{0.5x}$ | AutoReg |💯|👌|👌|👌|👌|👌|👌|👌📉|
| | | | OffsetModel |👌|👌|👌|👌|🤬|👌|👌|👌📈|
| |  | $2e^{0.5(2\pi - x)}$ | AutoReg |💯|👌|👌|👌|👌|👌|👌|👌📉|
| | | | OffsetModel |❌|👌|👌|👌|❗|👌|❗|❗📈|
| | Quadratic Trend | $x^2$ | AutoReg |💯|👌|👌|👌|👌|👌|👌|👌📈|
| | | | OffsetModel |👌|👌|👌|👌|🤬|👌|👌|👌📈|
| |  | $-x^2$ | AutoReg |💯|👌|👌|👌|👌|👌|👌|👌📉|
| | | | OffsetModel |👌|👌|👌|👌|🤬|👌|👌|👌📈|
| | Logarithmic Trend | $10 + 5\ln(x+1)$ | AutoReg |❗|👌|👌|👌|🤬|👌|👌|👌📈|
| | | | OffsetModel |✔️|👌|👌|👌|❗|👌|👌|👌📈|
| |  | $10 - 5\ln(x+1)$ | AutoReg |❗|👌|👌|👌|🤬|👌|💀|💀📉|
| | | | OffsetModel |✔️|👌|👌|👌|❗|👌|❗|❌📈|
| | Sigmoidal/Logistic Trend | $\frac{10}{1 + e^{-x + 5}}$ | AutoReg |🛑|💀|💀|💀|🤬|👌|❗|❗📈|
| | | | OffsetModel |👌|👌|👌|👌|🤬|👌|👌|👌📈|
| |  | $-\frac{10}{1 + e^{-x + 5}}$ | AutoReg |🛑|💀|💀|💀|🤬|👌|❗|❗📉|
| | | | OffsetModel |👌|👌|👌|👌|🤬|👌|👌|👌📈|
| | Seasonality | $20 \cdot \sin(4x) + 50$ | AutoReg |🛑|❗|☠|❌|🤬|👌|❌|❗📉|
| | | | OffsetModel |👌|👌|👌|👌|✔️|👌|👌|👌📈|
| | Outliers | ![outliers math func](https://raw.githubusercontent.com/ranggakd/DAIly/main/ideas/regression_forecasting_metrics/assets/outliers_formula_b70.png) | AutoReg |🛑|❗|☠|❗|🤬|☠|☠|☠📉|
| | | | OffsetModel |👌|👌|❌|👌|✔️|☠|☠|☠📈|
| | Repeated Patterns | $(x \mod 5) + 1$ | AutoReg |🛑|💀|💀|💀|❗|👌|💀|💀📈|
| | | | OffsetModel |👌|👌|👌|👌|👌|👌|👌|👌📈|
| Nature of Errors | Systematic Overestimation | $10 \cdot \cos(x)$ | OffsetModel 1% |👌|👌|👌|👌|❗|👌|❗|👌📈|
| | | | OffsetModel 10% |❌|✔️|❗|✔️|🤬|👌|💀|❗📈|
| | Systematic Underestimation | $10 \cdot \cos(x)$ | NegOffsetModel 1% |👌|👌|👌|👌|❗|👌|❗|👌📉|
| | | | NegOffsetModel 10% |❌|✔️|❗|❗|🤬|👌|💀|❗📉|
| | Random Errors | $10 \cdot \cos(x)$ | RandomOffsetModel 1% |👌|👌|👌|👌|❗|👌|❗|✔️📉|
| | | | RandomOffsetModel 10% |❌|✔️|❗|✔️|🤬|👌|💀|❌📈|

</details>

---